# Загрузка библиотек

In [72]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import joblib

# === 1. Загрузка данных ===

In [73]:
data = pd.read_csv("spam.csv", encoding="latin-1")
data = data[['v1', 'v2']]
data.columns = ['label', 'message']
data['label'] = data['label'].map({'ham': 0, 'spam': 1})

# === 2. Очистка текста ===

In [74]:
def clean_data(text):
    text = text.lower()                               # в нижний регистр
    text = re.sub(r"http\S+", " ", text)              # убираем ссылки
    text = re.sub(r"\d+", " ", text)                  # убираем числа
    text = text.translate(str.maketrans("", "", string.punctuation))  # убираем пунктуацию
    text = re.sub(r"\s+", " ", text).strip()          # убираем лишние пробелы
    return text

data['message'] = data['message'].apply(clean_data)

X = data['message']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Оптимизация KNN

In [75]:
knn_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('knn', KNeighborsClassifier())
])

param_grid = {
    'tfidf__max_features': [5000, 10000],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'knn__n_neighbors': [3, 5, 7],
    'knn__metric': ['cosine', 'euclidean'],
    'knn__weights': ['uniform', 'distance']
}

# === Обучение ===

In [76]:
knn_grid = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)
knn_grid.fit(X_train, y_train)

,estimator,Pipeline(step...lassifier())])
,param_grid,"{'knn__metric': ['cosine', 'euclidean'], 'knn__n_neighbors': [3, 5, ...], 'knn__weights': ['uniform', 'distance'], 'tfidf__max_features': [5000, 10000], ...}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


# === 5. Обучение k-NN ===

In [77]:
print("Лучшие параметры KNN:", knn_grid.best_params_)
y_pred = knn_grid.predict(X_test)
print("Точность:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Лучшие параметры KNN: {'knn__metric': 'cosine', 'knn__n_neighbors': 3, 'knn__weights': 'distance', 'tfidf__max_features': 10000, 'tfidf__ngram_range': (1, 2)}
Точность: 0.9766816143497757
F1-score: 0.9057971014492754

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.98      0.84      0.91       149

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



# === Сохранение модели ===

In [78]:
joblib.dump(knn_grid.best_estimator_, 'knn_spam_model.pkl')
print("\nМодель сохранена как 'knn_spam_model.pkl'")


Модель сохранена как 'knn_spam_model.pkl'


# Сохранение векторизатора отдельно (для возможности использования отдельно)

In [79]:
vectorizer = knn_grid.best_estimator_.named_steps['tfidf']
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("Векторизатор сохранен как 'tfidf_vectorizer.pkl'")

Векторизатор сохранен как 'tfidf_vectorizer.pkl'


# =============================================================================
# ОТДЕЛЬНАЯ ПРОВЕРКА МОДЕЛИ НА НОВЫХ ДАННЫХ
# =============================================================================

In [81]:
# Загрузка сохраненной модели
loaded_model = joblib.load('knn_spam_model.pkl')

# Примеры сообщений для проверки
test_messages = [
    "Congratulations! You won $1000 prize! Call now to claim.",  # спам
    "Hey, are we still meeting for lunch tomorrow?",  # не спам
    "URGENT: Your bank account needs verification. Click here.",  # спам
    "Hi mom, I'll be home late today. See you for dinner",  # не спам
    "FREE iPhone waiting for you! Claim your prize now!",  # спам
    "Meeting rescheduled to 3 PM. Please confirm attendance."  # не спам
]

In [82]:
# Предсказание для тестовых сообщений
print("\nПредсказания для тестовых сообщений:")
print("-" * 50)

for i, message in enumerate(test_messages, 1):
    # Очистка текста
    cleaned_message = clean_data(message)
    # Предсказание
    prediction = loaded_model.predict([cleaned_message])[0]
    probability = loaded_model.predict_proba([cleaned_message])[0]
    
    spam_prob = probability[1] * 100  # вероятность спама
    label = "СПАМ" if prediction == 1 else "НЕ СПАМ"
    
    print(f"{i}. Сообщение: {message[:50]}...")
    print(f"   Результат: {label} (вероятность спама: {spam_prob:.1f}%)")
    print()


Предсказания для тестовых сообщений:
--------------------------------------------------
1. Сообщение: Congratulations! You won $1000 prize! Call now to ...
   Результат: СПАМ (вероятность спама: 100.0%)

2. Сообщение: Hey, are we still meeting for lunch tomorrow?...
   Результат: НЕ СПАМ (вероятность спама: 0.0%)

3. Сообщение: URGENT: Your bank account needs verification. Clic...
   Результат: НЕ СПАМ (вероятность спама: 28.7%)

4. Сообщение: Hi mom, I'll be home late today. See you for dinne...
   Результат: НЕ СПАМ (вероятность спама: 0.0%)

5. Сообщение: FREE iPhone waiting for you! Claim your prize now!...
   Результат: СПАМ (вероятность спама: 67.1%)

6. Сообщение: Meeting rescheduled to 3 PM. Please confirm attend...
   Результат: НЕ СПАМ (вероятность спама: 0.0%)



In [83]:
# Проверка на тестовой выборке с использованием сохраненной модели
print("Проверка на исходной тестовой выборке:")
print("-" * 40)

# Преобразуем тестовые данные так же, как при обучении
X_test_cleaned = X_test.apply(clean_data)
y_pred_loaded = loaded_model.predict(X_test_cleaned)

print(f"Точность загруженной модели: {accuracy_score(y_test, y_pred_loaded):.4f}")
print(f"F1-score загруженной модели: {f1_score(y_test, y_pred_loaded):.4f}")

Проверка на исходной тестовой выборке:
----------------------------------------
Точность загруженной модели: 0.9767
F1-score загруженной модели: 0.9058


In [84]:
# Функция для интерактивной проверки
def check_spam_interactive():
    print("\n" + "="*50)
    print("ИНТЕРАКТИВНАЯ ПРОВЕРКА СПАМ-ФИЛЬТРА")
    print("="*50)
    print("Введите сообщение для проверки (или 'quit' для выхода):")
    
    while True:
        user_input = input("\nВаше сообщение: ")
        if user_input.lower() == 'quit':
            break
            
        if user_input.strip():
            # Очистка и предсказание
            cleaned_input = clean_data(user_input)
            prediction = loaded_model.predict([cleaned_input])[0]
            probability = loaded_model.predict_proba([cleaned_input])[0]
            
            spam_prob = probability[1] * 100
            if prediction == 1:
                print(f"🚫 СПАМ (вероятность: {spam_prob:.1f}%)")
            else:
                print(f"✅ НЕ СПАМ (вероятность спама: {spam_prob:.1f}%)")


In [ ]:
# Запуск интерактивной проверки
check_spam_interactive()

# Матрица ошибок для детального анализа
print("\nМатрица ошибок:")
cm = confusion_matrix(y_test, y_pred_loaded)
print(cm)
print("\n(0 - не спам, 1 - спам)")


ИНТЕРАКТИВНАЯ ПРОВЕРКА СПАМ-ФИЛЬТРА
Введите сообщение для проверки (или 'quit' для выхода):
✅ НЕ СПАМ (вероятность спама: 0.0%)
✅ НЕ СПАМ (вероятность спама: 0.0%)
✅ НЕ СПАМ (вероятность спама: 0.0%)
✅ НЕ СПАМ (вероятность спама: 33.3%)
✅ НЕ СПАМ (вероятность спама: 33.3%)
✅ НЕ СПАМ (вероятность спама: 0.0%)
✅ НЕ СПАМ (вероятность спама: 33.3%)
✅ НЕ СПАМ (вероятность спама: 33.3%)
✅ НЕ СПАМ (вероятность спама: 33.3%)
✅ НЕ СПАМ (вероятность спама: 33.3%)

ИНФОРМАЦИЯ О МОДЕЛИ
Размер словаря: 10000
Метрика: cosine
Количество соседей: 3
Вес: distance

Матрица ошибок:
[[964   2]
 [ 24 125]]

(0 - не спам, 1 - спам)
